In [ ]:
# Colab 전용: tqdm, pandas, matplotlib 설치/업데이트
!pip -q install pandas tqdm matplotlib

# (선택) 구글 드라이브 마운트: EDA 결과물만 저장하고, 대형 TSV는 /content 에 임시 저장 권장
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import os, json, pathlib

# kaggle.json 업로드 안내
from google.colab import files
print("kaggle.json 업로드 버튼을 눌러주세요.")
uploaded = files.upload()

os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'wb') as f:
    f.write(uploaded['kaggle.json'])
os.chmod('/root/.kaggle/kaggle.json', 0o600)

!pip -q install kaggle
!kaggle -h | head -n 3


kaggle.json 업로드 버튼을 눌러주세요.


Saving kaggle.json to kaggle.json
usage: kaggle [-h] [-v] [-W]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...


In [ ]:
KAGGLE_DATASET = "felixstahlberg/the-c4-200m-dataset-for-gec"  # <-- 실제 슬러그로 교체

# 목록만 먼저 본다(용량 파악, 파일명 확인)
!kaggle datasets files {KAGGLE_DATASET}


name                            size  creationDate                
------------------------  ----------  --------------------------  
edits.tsv-00000-of-00010  2992601947  2021-05-26 18:59:22.840000  
edits.tsv-00001-of-00010  2990782945  2021-05-26 19:00:11.393000  
edits.tsv-00002-of-00010  2991847440  2021-05-26 19:00:57.927000  
edits.tsv-00003-of-00010  2990847051  2021-05-26 19:01:38.295000  
edits.tsv-00004-of-00010  2991463256  2021-05-26 19:02:19.945000  
edits.tsv-00005-of-00010  2992038211  2021-05-26 19:03:00.175000  
edits.tsv-00006-of-00010  2992255660  2021-05-26 19:03:41.023000  
edits.tsv-00007-of-00010  2993397600  2021-05-26 19:04:18.859000  
edits.tsv-00008-of-00010  2991033963  2021-05-26 19:05:03.250000  
edits.tsv-00009-of-00010  2991637077  2021-05-26 19:05:42.223000  


In [ ]:
import subprocess, re, os, sys, textwrap

# 원하는 샤드 인덱스 지정 (예: 0, 1 두 개만)
TARGET_SHARDS = [2, 3,4,5,6,7,8,9]

# 파일 패턴 예상. 목록에서 실제 이름을 확인한 후 패턴을 맞추세요.
def shard_name(idx: int) -> str:
    return f"edits.tsv-{idx:05d}-of-00010"

# 어디에 저장할지 지정
download_dir = "/content/drive/MyDrive/LikeLion_NLP2/Project_2/c4_200m_edits"
os.makedirs(download_dir, exist_ok=True)

# 샤드별로 단건 다운로드
for i in TARGET_SHARDS:
    fname = shard_name(i)
    print(f"Downloading {fname} ...")
    # --force 사용해 동일 파일이 있어도 재시도 가능
    !kaggle datasets download {KAGGLE_DATASET} -f {fname} -p {download_dir} --force
    # Kaggle는 zip으로 떨어질 수 있음 -> 풀기
    zip_path = os.path.join(download_dir, fname + ".zip")
    if os.path.exists(zip_path):
        !unzip -o {zip_path} -d {download_dir}
        os.remove(zip_path)

!ls -lh {download_dir}


Dataset URL: https://www.kaggle.com/datasets/felixstahlberg/the-c4-200m-dataset-for-gec
License(s): Attribution 4.0 International (CC BY 4.0)
 99% 967M/975M [00:06<00:00, 177MB/s]
100% 975M/975M [00:06<00:00, 148MB/s]
Dataset URL: https://www.kaggle.com/datasets/felixstahlberg/the-c4-200m-dataset-for-gec
License(s): Attribution 4.0 International (CC BY 4.0)
100% 973M/975M [00:07<00:00, 178MB/s]
100% 975M/975M [00:07<00:00, 135MB/s]
Dataset URL: https://www.kaggle.com/datasets/felixstahlberg/the-c4-200m-dataset-for-gec
License(s): Attribution 4.0 International (CC BY 4.0)
 99% 969M/975M [00:08<00:00, 163MB/s]
100% 975M/975M [00:08<00:00, 115MB/s]
Dataset URL: https://www.kaggle.com/datasets/felixstahlberg/the-c4-200m-dataset-for-gec
License(s): Attribution 4.0 International (CC BY 4.0)
 98% 957M/975M [00:07<00:00, 146MB/s]
100% 975M/975M [00:07<00:00, 131MB/s]
Dataset URL: https://www.kaggle.com/datasets/felixstahlberg/the-c4-200m-dataset-for-gec
License(s): Attribution 4.0 Internationa

In [ ]:
from huggingface_hub import hf_hub_download
import os, shutil

# 경로지정 예시 '/C4/en/c4-train.00000-of-01024.json.gz'
# 구글 드라이브로 경로 지정해도 상관x
# 얼마나 받을지 range 숫자 조정
# 각 파일당 310MB정도
for s in [f"en/c4-train.{i:05d}-of-01024.json.gz" for i in range(0,2)]:
    local = hf_hub_download("allenai/c4", s, repo_type="dataset")
    dst = "/C4/en/" + os.path.basename(s)
    if not os.path.exists(dst):
        shutil.copy2(local, dst)
!ls -lh /C4/en | tail